In [9]:
import os
import glob
import subprocess
import time
import pandas as pd

In [10]:
songs = pd.read_csv(r'D:\Mafer\uni\BD2\proyecto_2_bd2-main\proyecto_2_bd2-main\backend\spotifyData.csv')
# songs = songs.head(3)
songs = songs.loc[8801:8850]
songs

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,tempo,duration_ms,language
8801,3yZAuRxvWRQHl79e7Vc6Vw,Better Than Yesterday - Club Mix,Sidney Samson,Hey. Hey. Hey. Eeeeyah oh. Eeeeyah oh Hey. Hey...,0,06WvKsEoWCEPNpaLcc2y5N,Better Than Yesterday,2013-01-07,Vocal House,5PCAWKfUWAUj8VeY8G7xRQ,edm,progressive electro house,127.958,309387,en
8802,3Yzcwfm3sEQaaa3ENFC18h,Against the World - Radio Edit,Morgan Page,I guess it's just me and you for now Against t...,23,1pkE0cQN1tHZ2LpFpp5JJJ,Against the World - Single,2013-11-19,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,127.948,228760,en
8803,3yZQk5PC52CCmT4ZaTIKvv,This Is Gospel,Panic! At The Disco,"NA This is gospel, for the fallen ones Locked ...",72,1hxraaWEf3wFnJxADf8Dge,"Too Weird To Live, Too Rare To Die!",2013-10-04,random playlist that can't decide its genre so...,49qyKvg6UAQAvIMUIGMnP3,pop,indie poptimism,156.073,187187,en
8804,3Z1Xc4igRjuW1jAw2BSm2d,Talvez Bem Mais,Clara Lima,Talvez bem mais do que eu deveria Longe demais...,39,5jFvIxCNBTGhrh4AkObWI5,Selfie,2019-11-22,Dose Trap,37i9dQZF1DWWlW7KLhEhCZ,rap,trap,149.943,205924,pt
8805,3z4CGd63tpUn9a6oQSG0CI,Location (feat. Burna Boy),Dave,JAE5 If you send me the location Then I'll be ...,79,4GrFuXwRmEBJec22p58fsD,PSYCHODRAMA,2019-03-08,Hip-Hop Mixtape,37i9dQZF1DWVdgXTbYm2r0,rap,hip hop,109.979,241293,en
8806,3Z6O6zNyxqmqevwy5Dy4cs,Out Of Sight,Logic,"What? We gon' make it, knock on wood We gon' m...",7,6GeHCNwwqMMUrpxuGTRYcf,Confessions of a Dangerous Mind,2019-05-10,This Is Logic,37i9dQZF1DWTIuAboZgTMf,rap,hip hop,174.022,137947,en
8807,3z7WmSgmSDTFH1WwL18yyS,Bad Dracula,Red Velvet,보름달 아래 날 마주치면 딱 알아봐 놀래 킬 생각은 전혀 없어 없어 Baby 오늘 ...,0,2pBrB5hW9CjLWWZGqrET2g,Russian Roulette - The 3rd Mini Album,2016-09-07,K-Crazy Michioso Tunes,37i9dQZF1DWUXxc8Mc6MmJ,edm,pop edm,93.986,188331,ko
8808,3Z9c2ZPX73VO5wnbqbrz3j,Hypnotic Energy,Liquid Soul,"Listen, can you hear it? The music I can here ...",18,36Y0XTd0yxsK8mhkqVN6ww,Hypnotic Energy Remixes,2015-08-28,Underground Party | Hypnotic | Minimal | Acid ...,29jj7pQlDqnWclbHQk21Rq,edm,progressive electro house,134.973,510320,en
8809,3Z9mE6hpkwil8jogHOy4Lk,Tonight I'm Gonna Rock You Tonight,Spinal Tap,"Little girl, it's a great big world But there'...",0,6S5mWLewzhLaAPmgDsnLlM,This Is Spinal Tap (Soundtrack),1984-01-01,"Classic Rock 70s 80s 90s, Rock Classics - 70s ...",3Ho3iO0iJykgEQNbjB2sic,rock,classic rock,92.132,156267,en
8810,3Z9vR8JrHFyyjB0JoHzx6p,Pretty,Landon Cube,"Yeah, my bitch like girls Think I might die yo...",62,6cnC79HOafGgH2sUT7zsnh,Pretty,2019-10-18,Sad Trap,7kzKtw5vug3IIxKfLadi8A,rap,trap,76.064,191440,en


In [ ]:
def get_song_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, stderr=subprocess.PIPE, text=True, encoding='utf-8')
    output = result.stderr
    duration_line = [line for line in output.splitlines() if "Duration" in line]
    if duration_line:
        duration_str = duration_line[0].split(",")[0].split("Duration:")[1].strip()
        h, m, s = map(float, duration_str.split(":"))
        return int(h * 3600 + m * 60 + s)
    return 0


In [ ]:
def get_and_trim_song(track_id, max_retries):
    url = f'https://open.spotify.com/track/{track_id}'
    song_directory = f'songs/{track_id}'
    os.makedirs(song_directory, exist_ok=True)  

    try_request = 0
    while try_request < max_retries:
        print(f"Intento {try_request + 1}/{max_retries} para {track_id}...")
        command_download = ['spotdl', 'download', url, '--output', song_directory]
        try_request += 1
        try:
            subprocess.run(command_download, check=True)

            downloaded_songs = glob.glob(os.path.join(song_directory, '*.mp3'))
            if downloaded_songs:
                input_file = downloaded_songs[0]
                total_seconds = get_song_duration(input_file)
                if total_seconds == 0:
                    print(f"No se pudo obtener la duración de la canción {track_id}.")
                    return False

                half_duration = total_seconds // 2  

                output_file = f'{song_directory}/{track_id}.mp3'
                command_trim = [
                    'ffmpeg',
                    '-i', input_file,  
                    '-ss', str(half_duration - 15),  
                    '-t', '30',  
                    '-c', 'copy', 
                    output_file 
                ]
                subprocess.run(command_trim, check=True)

                if os.path.exists(output_file):
                    print(f"Archivo recortado guardado: {output_file}")
                    return True
        except subprocess.CalledProcessError as e:
            print(f"Error en el intento {try_request + 1}: {e}")
            time.sleep(1)  
            try_request += 1
    print(f"Error al descargar o recortar la canción {track_id}.")
    return False

for track_id in songs['track_id']:
    song_directory = os.path.join("songs", track_id)

    downloaded_songs = glob.glob(os.path.join(song_directory, '*.mp3'))
    if not downloaded_songs:
        if not get_and_trim_song(track_id, 3):
            print(f"Error en la descarga de {track_id}.")
    else:
        print(f'{track_id} ya existe.')

Intento 1/3 para 3yZAuRxvWRQHl79e7Vc6Vw...
Archivo recortado guardado: songs/3yZAuRxvWRQHl79e7Vc6Vw/3yZAuRxvWRQHl79e7Vc6Vw.mp3
Intento 1/3 para 3Yzcwfm3sEQaaa3ENFC18h...
Archivo recortado guardado: songs/3Yzcwfm3sEQaaa3ENFC18h/3Yzcwfm3sEQaaa3ENFC18h.mp3
Intento 1/3 para 3yZQk5PC52CCmT4ZaTIKvv...
Archivo recortado guardado: songs/3yZQk5PC52CCmT4ZaTIKvv/3yZQk5PC52CCmT4ZaTIKvv.mp3
Intento 1/3 para 3Z1Xc4igRjuW1jAw2BSm2d...
Archivo recortado guardado: songs/3Z1Xc4igRjuW1jAw2BSm2d/3Z1Xc4igRjuW1jAw2BSm2d.mp3
Intento 1/3 para 3z4CGd63tpUn9a6oQSG0CI...
Archivo recortado guardado: songs/3z4CGd63tpUn9a6oQSG0CI/3z4CGd63tpUn9a6oQSG0CI.mp3
Intento 1/3 para 3Z6O6zNyxqmqevwy5Dy4cs...
Archivo recortado guardado: songs/3Z6O6zNyxqmqevwy5Dy4cs/3Z6O6zNyxqmqevwy5Dy4cs.mp3
Intento 1/3 para 3z7WmSgmSDTFH1WwL18yyS...
Archivo recortado guardado: songs/3z7WmSgmSDTFH1WwL18yyS/3z7WmSgmSDTFH1WwL18yyS.mp3
Intento 1/3 para 3Z9c2ZPX73VO5wnbqbrz3j...
Archivo recortado guardado: songs/3Z9c2ZPX73VO5wnbqbrz3j/3Z9c2ZPX73V